In [ ]:
import pandas as pd
from datetime import datetime
import os
import csv

# Simulated accuracy check (for demo)
def check_accuracy(df, column, valid_values):
    if column not in df.columns:
        return None
    valid = df[column].isin(valid_values).sum()
    total = df[column].notnull().sum()
    return round((valid / total) * 100, 2) if total > 0 else None

# Completeness check
def check_completeness(df, columns):
    total_cells = df[columns].shape[0] * len(columns)
    missing_cells = df[columns].isnull().sum().sum()
    return round((1 - missing_cells / total_cells) * 100, 2) if total_cells > 0 else None

# Main monitoring function
def monitor_data_quality(data_path, log_path, completeness_fields, accuracy_checks):
    df = pd.read_csv(data_path)

    # Compute metrics
    completeness_score = check_completeness(df, completeness_fields)
    accuracy_score = sum([
        check_accuracy(df, col, valid) for col, valid in accuracy_checks.items()
        if check_accuracy(df, col, valid) is not None
    ]) / len(accuracy_checks)

    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

    # Ensure log file exists
    if not os.path.exists(log_path):
        with open(log_path, 'w', newline='') as f:
            writer = csv.writer(f)
            writer.writerow(["timestamp", "completeness", "accuracy"])

    # Append log entry
    with open(log_path, 'a', newline='') as f:
        writer = csv.writer(f)
        writer.writerow([timestamp, completeness_score, accuracy_score])

    print(f"[{timestamp}] Logged - Completeness: {completeness_score}%, Accuracy: {accuracy_score}%")

# Example usage
if _name_ == "_main_":
    monitor_data_quality(
        data_path="your_dataset.csv",
        log_path="data_quality_log.csv",
        completeness_fields=["id", "amount", "date"],
        accuracy_checks={
            "status": ["complete", "pending", "failed"],
            "currency": ["USD", "EUR", "GBP"]
        }
    )